Importing necessary libraries

In [70]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

Scraping Titles and HREF's

In [37]:
limit = 0
limits = '0'
pages = 20
href = []
titles = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'}

for p in range(pages):

    url = 'https://myanimelist.net/topanime.phphttps://myanimelist.net/anime/52991/topanime.php?limit=limits'
    page = requests.get(url,headers=headers)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content,'html.parser')
        links = soup.find_all('h3', class_='fl-l fs14 fw-b anime_ranking_h3')

        for h3 in links:
            a_tag = h3.find('a')
            if a_tag:
                href.append(a_tag.get('href'))
                titles.append(a_tag.get_text())

    else:
        print('Response not 200')
        break
    
    limit +=50
    limits = str(limit)
    



In [45]:
print(len(titles))

1000


In [46]:
def extract_info(newSoup):
    pass

Scraping other information

In [57]:
scores = []
description = []
rank = []
popularity = []
episodes = []
status = []
aired = []
premiered = []
source = []
genres = []
rating = []


def allPages(href):

    for hr in href:
        newPage = requests.get(hr,headers=headers)
        newSoup = BeautifulSoup(newPage.content,'html.parser')
        newSoup.prettify()
        sc,desc,r,pop,ep,stat,air,prem,src,gen,rat =  extract_info(newSoup)

        
        sc = float(sc)
        r = r.replace('Ranked#','')
        r = float(r)
        pop = pop.replace('Popularity#','')
        pop = int(pop)
        if ep != 'Unknown':
            ep = int(ep)
        else:
            ep = '0'
            ep = int(ep)
                
        if(prem != None):
            prem = prem.split(' ',1)[-1]
            prem = int(prem)
  #      print(sc)
  #      print(r)
  #      print(pop)
  #      print(ep)
  #      print(prem)
  #      print(desc)
  #      print(stat)
  #      print(air)
  #      print(gen)
  #      print(src)
  #      print(rat)
        scores.append(sc)
        description.append(desc)
        rank.append(r)
        popularity.append(pop)
        episodes.append(ep)
        status.append(stat)
        aired.append(air)
        premiered.append(prem)
        source.append(src)
        genres.append(gen)
        rating.append(rat)        
        

In [58]:
def extract_info(newSoup):
    description = None
    rank = None
    popularity = None
    episodes = None
    status = None
    aired = None
    premiered = None
    source = None
    genres = None
    rating = None

    # Extract score if present
    score = newSoup.find('div', class_='score-label')
    score_text = score.get_text(strip=True) if score else "Score not found"

    # Extract rank if present
    rankt = newSoup.find('span', class_='numbers ranked')
    rank = rankt.get_text(strip=True) if rankt else "Rank not found"

    # Extract popularity if present
    popularityp = newSoup.find('span', class_='numbers popularity')
    popularity = popularityp.get_text(strip=True) if popularityp else "Popularity not found"

    # Extract description if present
    descriptiond = newSoup.find('p', itemprop='description')
    description = descriptiond.get_text(strip=True) if descriptiond else "Description not found"

    allDivs = newSoup.find_all('div', class_='spaceit_pad')

    for div in allDivs:
        div_text = div.get_text(strip=True)
        
        if 'Episodes' in div_text:
            episodes = div_text.replace('Episodes:', '').strip()
        
        elif 'Status' in div_text:
            status = div_text.replace('Status:', '').strip()
        
        elif 'Aired' in div_text:
            aired = div_text.replace('Aired:', '').strip()
    
        elif 'Premiered' in div_text:
            premiered = div_text.replace('Premiered:', '').strip()
        
        elif 'Source' in div_text:
            source = div_text.replace('Source:', '').strip()
        
        elif 'Genres' in div_text:
            # Extract genres from <a> tags within the div
            genres = [a_tag.get_text(strip=True) for a_tag in div.find_all('a')]
            genres = ', '.join(genres)
        
        elif 'Rating' in div_text:
            rating = div_text.replace('Rating:', '').strip()
    
    return score_text,description, rank, popularity, episodes, status, aired, premiered, source, genres, rating


In [59]:
print(len(titles))
allPages(href)

1000


Created a pandas dataframe and then stored in .csv file

In [98]:
df = pd.DataFrame({'Rank':rank,'Link':href,'Title':titles,'Score':scores,'Description':description,'Year Premiered':premiered,'Episodes':episodes,'Popularity':popularity,'Genre':genres,'Status':status,'Source':source,'Rating':rating,'Aired':aired})
print(df.shape)



(1000, 13)


Sentiment Analysis of Description

In [99]:
sia = SentimentIntensityAnalyzer()
i = 0
sentimentScore = []
for desc in description:
    sentimentScore.append(sia.polarity_scores(desc))

df2 = pd.DataFrame(sentimentScore)
print(df2.head())

     neg    neu    pos  compound
0  0.055  0.796  0.149    0.9432
1  0.175  0.755  0.070   -0.9690
2  0.138  0.719  0.143   -0.0015
3  0.131  0.755  0.114   -0.5930
4  0.107  0.856  0.037   -0.8221


Merging both dataframes into one and then saving it

In [100]:
# Reset index to use it as a column for merging
df1_reset = df.reset_index()
df2_reset = df2.reset_index()

# Merge DataFrames on the index column
final = pd.merge(df1_reset, df2_reset, on='index', how='left')

# Drop the extra index column if desired
final = final.drop(columns='index')

# Save to CSV
final.to_csv('Anime_Data.csv', index=False)